In [1]:
# Dans un terminal ou une cellule de notebook
! pip install tabula-py camelot-py opencv-python pandas

In [2]:
! pip install pytesseract pdf2image PyPDF2

In [4]:
import os
import tabula
import pandas as pd
import camelot
import pytesseract
from pdf2image import convert_from_path
import PyPDF2
import re

def clean_cid_markers(text):
    """Nettoie les marqueurs CID des textes extraits des PDFs"""
    return re.sub(r'\(cid:\d+\)', '', text)

def extract_pdf_tables(pdf_path: str, output_dir: str = "extracted_tables"):
    """
    Extract tables from a PDF file using multiple libraries for best results.
    """
    if not os.path.exists(pdf_path):
        print(f"Erreur: Fichier non trouvé: {pdf_path}")
        return
    
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Extraction des tableaux de {pdf_path}...")
    
    # 1. Essayer avec tabula
    try:
        print("Utilisation de tabula...")
        tabula_tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
        
        if tabula_tables:
            print(f"Tabula a trouvé {len(tabula_tables)} tableaux.")
            
            # Sauvegarder chaque tableau
            for i, df in enumerate(tabula_tables):
                # Nettoyer les données
                df = df.applymap(lambda x: clean_cid_markers(str(x)) if isinstance(x, str) else x)
                
                output_path = os.path.join(output_dir, f"tabula_table_{i}.csv")
                df.to_csv(output_path, index=False)
                print(f"Tableau sauvegardé: {output_path}")
        else:
            print("Tabula n'a trouvé aucun tableau.")
    except Exception as e:
        print(f"Erreur avec tabula: {str(e)}")
    
    # 2. Essayer avec camelot avec différentes saveurs
    try:
        print("\nUtilisation de camelot (mode lattice)...")
        camelot_tables_lattice = camelot.read_pdf(pdf_path, pages='all', flavor='lattice')
        
        if len(camelot_tables_lattice) > 0:
            print(f"Camelot (lattice) a trouvé {len(camelot_tables_lattice)} tableaux.")
            
            # Sauvegarder chaque tableau
            for i, table in enumerate(camelot_tables_lattice):
                # Nettoyer les données
                df = table.df.applymap(lambda x: clean_cid_markers(str(x)) if isinstance(x, str) else x)
                
                output_path = os.path.join(output_dir, f"camelot_lattice_table_{i}.csv")
                df.to_csv(output_path, index=False)
                print(f"Tableau sauvegardé: {output_path}")
        else:
            print("Camelot (lattice) n'a trouvé aucun tableau.")
        
        print("\nUtilisation de camelot (mode stream)...")
        camelot_tables_stream = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
        
        if len(camelot_tables_stream) > 0:
            print(f"Camelot (stream) a trouvé {len(camelot_tables_stream)} tableaux.")
            
            # Sauvegarder chaque tableau
            for i, table in enumerate(camelot_tables_stream):
                # Nettoyer les données
                df = table.df.applymap(lambda x: clean_cid_markers(str(x)) if isinstance(x, str) else x)
                
                output_path = os.path.join(output_dir, f"camelot_stream_table_{i}.csv")
                df.to_csv(output_path, index=False)
                print(f"Tableau sauvegardé: {output_path}")
        else:
            print("Camelot (stream) n'a trouvé aucun tableau.")
    except Exception as e:
        print(f"Erreur avec camelot: {str(e)}")
    
    # 3. Essayer avec PyPDF2 pour extraction de texte brut
    try:
        print("\nUtilisation de PyPDF2 pour extraction de texte...")
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num, page in enumerate(reader.pages):
                page_text = page.extract_text()
                text += f"\n--- PAGE {page_num+1} ---\n" + page_text
        
        # Nettoyer les marqueurs CID
        text = clean_cid_markers(text)
        
        # Sauvegarder le texte complet
        output_path = os.path.join(output_dir, "pdf_text_extraction.txt")
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"Texte brut sauvegardé: {output_path}")
    except Exception as e:
        print(f"Erreur avec PyPDF2: {str(e)}")
    
    # 4. Essayer avec OCR
    try:
        print("\nUtilisation de l'OCR (Tesseract)...")
        images = convert_from_path(pdf_path)
        
        # Sauvegarder le texte OCR page par page
        for i, img in enumerate(images):
            ocr_text = pytesseract.image_to_string(img)
            
            # Essayer de détecter des tableaux dans l'image
            ocr_data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DATAFRAME)
            
            # Sauvegarder le texte OCR
            output_path = os.path.join(output_dir, f"ocr_page_{i+1}.txt")
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(ocr_text)
            
            # Sauvegarder les données OCR structurées
            output_path = os.path.join(output_dir, f"ocr_data_page_{i+1}.csv")
            ocr_data.to_csv(output_path, index=False)
            
        print(f"OCR terminé pour {len(images)} pages.")
    except Exception as e:
        print(f"Erreur avec OCR: {str(e)}")
    
    print("\nExtraction terminée. Vérifiez le dossier", output_dir)

# Fonction pour afficher un aperçu des tableaux extraits
def preview_extracted_tables(directory):
    """Affiche un aperçu des tableaux extraits"""
    # Fichiers CSV
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    
    if csv_files:
        print(f"Aperçu des {len(csv_files)} fichiers CSV extraits:")
        
        for csv_file in csv_files:
            file_path = os.path.join(directory, csv_file)
            try:
                df = pd.read_csv(file_path)
                print("\n" + "="*50)
                print(f"Fichier: {csv_file}")
                print(f"Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
                print("-"*50)
                print(df.head(3))  # Affiche les 3 premières lignes
            except Exception as e:
                print(f"Erreur lors de la lecture de {csv_file}: {str(e)}")
    else:
        print("Aucun fichier CSV trouvé dans le répertoire.")
    
    # Fichiers texte
    txt_files = [f for f in os.listdir(directory) if f.endswith('.txt')]
    
    if txt_files:
        print(f"\nAperçu des {len(txt_files)} fichiers texte extraits:")
        
        for txt_file in txt_files:
            file_path = os.path.join(directory, txt_file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read(500)  # Lire les 500 premiers caractères
                
                print("\n" + "="*50)
                print(f"Fichier: {txt_file}")
                print("-"*50)
                print(f"{content}...")
                print("\n[texte tronqué]")
            except Exception as e:
                print(f"Erreur lors de la lecture de {txt_file}: {str(e)}")
    else:
        print("Aucun fichier texte trouvé dans le répertoire.")

# Post-traitement pour créer une structure de données propre à partir des différentes extractions
def post_process_extractions(directory, output_file="processed_tables.csv"):
    """
    Analyse toutes les extractions et tente de créer une version consolidée et propre
    """
    print("\nPost-traitement des extractions...")
    
    # Liste tous les fichiers
    all_files = os.listdir(directory)
    csv_files = [f for f in all_files if f.endswith('.csv')]
    txt_files = [f for f in all_files if f.endswith('.txt')]
    
    # Identifier la meilleure extraction (heuristique simple: celle avec le plus de lignes et colonnes)
    best_csv = None
    max_cells = 0
    
    for csv_file in csv_files:
        try:
            df = pd.read_csv(os.path.join(directory, csv_file))
            num_cells = df.shape[0] * df.shape[1]
            
            if num_cells > max_cells:
                max_cells = num_cells
                best_csv = csv_file
        except:
            continue
    
    if best_csv:
        print(f"Meilleure extraction CSV identifiée: {best_csv}")
        # Charger et nettoyer la meilleure extraction
        best_df = pd.read_csv(os.path.join(directory, best_csv))
        
        # Nettoyage de base
        best_df = best_df.dropna(how='all')  # Supprimer les lignes vides
        best_df = best_df.dropna(axis=1, how='all')  # Supprimer les colonnes vides
        
        # Sauvegarder le résultat traité
        output_path = os.path.join(directory, output_file)
        best_df.to_csv(output_path, index=False)
        print(f"Table traitée sauvegardée dans: {output_path}")
        
        return best_df
    else:
        print("Aucune extraction CSV valide n'a été trouvée.")
        return None

# Exemple d'utilisation
if __name__ == "__main__":
    pdf_paths = [
        r"C:\Users\kosmo\pycode\MCP_Rag_pdf_tab2\sdata201635.pdf"
        # Ajoutez d'autres chemins si nécessaire
    ]
    
    output_dir = "tableaux_extraits"
    
    # Parcourir chaque PDF dans la liste
    for i, pdf_path in enumerate(pdf_paths):
        print(f"\nTraitement du PDF {i+1}/{len(pdf_paths)}: {pdf_path}")
        
        # Créer un sous-dossier pour chaque PDF pour éviter les écrasements
        pdf_name = os.path.basename(pdf_path).replace(".pdf", "")
        pdf_output_dir = os.path.join(output_dir, pdf_name)
        
        # Extraire les tableaux
        extract_pdf_tables(pdf_path, pdf_output_dir)
        
        # Afficher un aperçu
        preview_extracted_tables(pdf_output_dir)
        
        # Post-traitement
        processed_df = post_process_extractions(pdf_output_dir)


Traitement du PDF 1/1: C:\Users\kosmo\pycode\MCP_Rag_pdf_tab2\sdata201635.pdf
Extraction des tableaux de C:\Users\kosmo\pycode\MCP_Rag_pdf_tab2\sdata201635.pdf...
Utilisation de tabula...
Erreur avec tabula: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`

Utilisation de camelot (mode lattice)...
Camelot (lattice) a trouvé 7 tableaux.
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_lattice_table_0.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_lattice_table_1.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_lattice_table_2.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_lattice_table_3.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_lattice_table_4.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_lattice_table_5.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_lattice_table_6.csv

Utilisation de camelot (mode stream)...


C:\Users\kosmo\AppData\Local\Temp\ipykernel_20348\2751630715.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = table.df.applymap(lambda x: clean_cid_markers(str(x)) if isinstance(x, str) else x)


Camelot (stream) a trouvé 14 tableaux.
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_0.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_1.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_2.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_3.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_4.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_5.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_6.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_7.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_8.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_9.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_10.csv
Tableau sauvegardé: tableaux_extraits\sdata201635\camelot_stream_table_11.csv
Tableau sauvegardé: tableaux_extrai

C:\Users\kosmo\AppData\Local\Temp\ipykernel_20348\2751630715.py:76: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = table.df.applymap(lambda x: clean_cid_markers(str(x)) if isinstance(x, str) else x)


Texte brut sauvegardé: tableaux_extraits\sdata201635\pdf_text_extraction.txt

Utilisation de l'OCR (Tesseract)...
Erreur avec OCR: Unable to get page count. Is poppler installed and in PATH?

Extraction terminée. Vérifiez le dossier tableaux_extraits\sdata201635
Aperçu des 21 fichiers CSV extraits:

Fichier: camelot_lattice_table_0.csv
Dimensions: 1 lignes × 2 colonnes
--------------------------------------------------
                                                   0  \
0  Design Type(s)\nMeasurement Type(s)\nTechnolog...   

                                                   1  
0  data integration objective\n\n\n\nDemographics...  

Fichier: camelot_lattice_table_1.csv
Dimensions: 3 lignes × 3 colonnes
--------------------------------------------------
    0                     1   2
0 NaN                   NaN NaN
1 NaN  MIMIC-III\n Database NaN
2 NaN                   NaN NaN

Fichier: camelot_lattice_table_2.csv
Dimensions: 13 lignes × 1 colonnes
------------------------------